In [4]:
import numpy as np

a = np.array([[1], [3]])
b = np.array([[5], [7]])
c = np.dstack([a, b])  # 각행의 행들을 열방향으로 배열 결합

print(a)
print(b)
print("--------")
print(c)

[[1]
 [3]]
[[5]
 [7]]
--------
[[[1 5]]

 [[3 7]]]


In [2]:
# %autosave 0
import numpy as np
import tensorflow
import tensorflow as tf

# tensorflow 2.0에 내장된 Keras 사용
from tensorflow.keras.models import Sequential  # class
from tensorflow.keras.models import load_model  # model 사용 함수
from tensorflow.keras.layers import Dense       # 전결합층
from tensorflow.keras.optimizers import Adam    # 가중치, bias 최적화
from tensorflow.keras.callbacks import Callback

# tensorflow 1.x, Keras가 독립적으로 설치된 경우
# from keras.models import Sequential  # class
# from keras.models import load_model  # model 사용 함수
# from keras.layers import Dense       # class

print(tf.__version__)

2.4.1


In [3]:
# 데이터
x_train = np.array([i for i in range(1, 21)])

# 정답(Target, 실제값)
y_train = np.array([i for i in range(2, 42, 2)])

print(x_train)
print(y_train)

[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20]
[ 2  4  6  8 10 12 14 16 18 20 22 24 26 28 30 32 34 36 38 40]


In [5]:
from tensorflow.keras.callbacks import Callback

# Callback class: 훈련중에 개발자가 제작한 특정 함수등을 실행 가능
# Callback class를 상속받아 GetWeights class를 구현
class GetWeights(Callback):
    # 가중치와 Bias를 저장할 Dictionary 선언 
    def __init__(self):
        super(GetWeights, self).__init__()
        self.weight_dict = {}

    # 각각의 학습이 끝난 후 실행       
    def on_epoch_end(self, epoch, logs=None):
        
        # Layer 수만큼 순환 하면서 weights, biases 수집
        for layer_i in range(len(self.model.layers)):
            w = self.model.layers[layer_i].get_weights()[0] # 가중치
            b = self.model.layers[layer_i].get_weights()[1] # 편향
            print('Layer %s has weights of shape %s and biases of shape %s' %(layer_i, np.shape(w), np.shape(b)))
            print('Epoch:', epoch)

            # 첫번째 학습(0)이라면 np.dstack 함수를 사용하지 말고 dictionary에 weights, biases 저장
            if epoch == 0:
                self.weight_dict['w_'+str(layer_i+1)] = w
                self.weight_dict['b_'+str(layer_i+1)] = b
                
            else: # 두번째 학습부터 이전에 만든 가중치 배열에 새 가중치를 추가
                self.weight_dict['w_'+str(layer_i+1)] = np.dstack((self.weight_dict['w_'+str(layer_i+1)], w))
                self.weight_dict['b_'+str(layer_i+1)] = np.dstack((self.weight_dict['b_'+str(layer_i+1)], b))

### 1 epoch, 전체 데이터 1회 학습

In [7]:
tf.random.set_seed(0) # 가중치, 편향이 일정하게 변경됨
model = Sequential()
model.add(Dense(3, input_dim=1, activation='linear'))
model.add(Dense(2, activation='linear'))
model.add(Dense(1, activation='linear'))

model.compile(optimizer=Adam(lr=0.001), loss='mse')
model.summary()

gw = GetWeights()
hist = model.fit(x_train, y_train, validation_split=0.2, shuffle=True,
                 epochs=1, batch_size=1, callbacks=[gw])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 3)                 6         
_________________________________________________________________
dense_4 (Dense)              (None, 2)                 8         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 3         
Total params: 17
Trainable params: 17
Non-trainable params: 0
_________________________________________________________________
16/16 [==============================] - 1s 32ms/step - loss: 219.2789 - val_loss: 778.6681
Layer 0 has weights of shape (1, 3) and biases of shape (3,)
Epoch: 0
Layer 1 has weights of shape (3, 2) and biases of shape (2,)
Epoch: 0
Layer 2 has weights of shape (2, 1) and biases of shape (1,)
Epoch: 0


In [8]:
print(type(gw.weight_dict))
print(gw.weight_dict)
# {
#     첫번재 층의 가중치들
#     'w_1': array([[ -0.522119  , -0.7313916 ,  0.07418509]], dtype=float32),
#     'b_1': array([-0.01457292, -0.01463656, -0.0145136], dtype=float32),
    
#     두번재 층의 가중치들
#     'w_2': array([[0.13398203, -0.62317073],
#                   [0.812081  ,  0.59494907],
#                   [0.30550095,  0.07893033]], dtype=float32),
#     'b_2': array([-0.01456596,  0.01454437], dtype=float32), 
    
#     세번재 층의 가중치들
#     'w_3': array([[-0.8746521 ], [ 0.66687274]], dtype=float32), 
#     'b_3': array([0.01455653], dtype=float32)
# }

<class 'dict'>
{'w_1': array([[-0.522119  , -0.7313916 ,  0.07418509]], dtype=float32), 'b_1': array([-0.01457292, -0.01463656, -0.0145136 ], dtype=float32), 'w_2': array([[ 0.13398203, -0.62317073],
       [ 0.812081  ,  0.59494907],
       [ 0.30550095,  0.07893033]], dtype=float32), 'b_2': array([-0.01456596,  0.01454437], dtype=float32), 'w_3': array([[-0.8746521 ],
       [ 0.66687274]], dtype=float32), 'b_3': array([0.01455653], dtype=float32)}


In [9]:
weight = gw.weight_dict["w_1"]
print(weight[0][0])

-0.522119


In [10]:
for key in gw.weight_dict: # 키 산출
    print(str(key) + ' shape: %s' %str(np.shape(gw.weight_dict[key])))  # 면, 행, 열
    print(gw.weight_dict[key])
    print('---------------------------------------')

w_1 shape: (1, 3)
[[-0.522119   -0.7313916   0.07418509]]
---------------------------------------
b_1 shape: (3,)
[-0.01457292 -0.01463656 -0.0145136 ]
---------------------------------------
w_2 shape: (3, 2)
[[ 0.13398203 -0.62317073]
 [ 0.812081    0.59494907]
 [ 0.30550095  0.07893033]]
---------------------------------------
b_2 shape: (2,)
[-0.01456596  0.01454437]
---------------------------------------
w_3 shape: (2, 1)
[[-0.8746521 ]
 [ 0.66687274]]
---------------------------------------
b_3 shape: (1,)
[0.01455653]
---------------------------------------


### 4 epoch, 전체 데이터 4회 학습

In [11]:
tf.random.set_seed(0)
model = Sequential()
model.add(Dense(3, input_dim=1, activation='linear'))
model.add(Dense(2, activation='linear'))
model.add(Dense(1, activation='linear'))

model.compile(optimizer=Adam(lr=0.001), loss='mse')
model.summary()

gw = GetWeights()
hist = model.fit(x_train, y_train, validation_split=0.2, shuffle=True,
                 epochs=4, batch_size=1, callbacks=[gw])

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 3)                 6         
_________________________________________________________________
dense_7 (Dense)              (None, 2)                 8         
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 3         
Total params: 17
Trainable params: 17
Non-trainable params: 0
_________________________________________________________________
Epoch 1/4
16/16 [==============================] - 0s 11ms/step - loss: 219.2789 - val_loss: 778.6681
Layer 0 has weights of shape (1, 3) and biases of shape (3,)
Epoch: 0
Layer 1 has weights of shape (3, 2) and biases of shape (2,)
Epoch: 0
Layer 2 has weights of shape (2, 1) and biases of shape (1,)
Epoch: 0
Epoch 2/4
16/16 [==============================] - 0s 4ms/step - loss: 212.1

In [12]:
for key in gw.weight_dict:
    print(str(key) + ' shape: %s' %str(np.shape(gw.weight_dict[key])))  # 면, 행, 열
    print(gw.weight_dict[key])
    print('---------------------------------------')

w_1 shape: (1, 3, 4)
[[[-0.522119   -0.53371495 -0.54630625 -0.558742  ]
  [-0.7313916  -0.74330246 -0.7565842  -0.77003616]
  [ 0.07418509  0.0629003   0.05105627  0.03978945]]]
---------------------------------------
b_1 shape: (1, 3, 4)
[[[-0.01457292 -0.02811802 -0.04227672 -0.05645445]
  [-0.01463656 -0.02855383 -0.04352024 -0.05890631]
  [-0.0145136  -0.02769074 -0.04098279 -0.05377849]]]
---------------------------------------
w_2 shape: (3, 2, 4)
[[[ 0.13398203  0.14563543  0.15835534  0.1709879 ]
  [-0.62317073 -0.63471246 -0.64716756 -0.6593824 ]]

 [[ 0.812081    0.82370824  0.83637     0.8489152 ]
  [ 0.59494907  0.58343387  0.571038    0.55891186]]

 [[ 0.30550095  0.29461062  0.2838476   0.27459118]
  [ 0.07893033  0.08970375  0.10018678  0.10904118]]]
---------------------------------------
b_2 shape: (1, 2, 4)
[[[-0.01456596 -0.02806749 -0.04212009 -0.05611625]
  [ 0.01454437  0.02791271  0.04165311  0.05514985]]]
---------------------------------------
w_3 shape: (2, 1